In [1]:
import sys
import logging
import json
import numpy as np
import pandas as pd
from datetime import datetime
import time

from apsopt.core.optimizer import Optimizer, get_generator_class
from apsopt.core.evaluators import FunctionEvaluator
from apsopt.util.pydantic.options import AlgorithmOptions
from apsopt.util.pydantic.vocs import GVOCS
import sext_tools as st
from gen3_shared import prepare, get_eval_f, compute_family_k2l_from_knob_k2l, get_raw_eval_f

logger = logging.getLogger(__name__)

### SETUP

In [3]:
MODE = 'DW_SH1N_SH3N_SH4N_SL1_SL2_SL3_SH1DW081828_SH3DW081828_SH4DW081828'
NAME_ROOT = f'mobo_{MODE}_v1'

from apsopt.util.log import prepare_loggers
prepare_loggers()
fh = logging.FileHandler(f'logs/{NAME_ROOT}_{datetime.now():%Y%m%dT%H%M%S}.optlog', mode='w')
fh.setLevel(logging.NOTSET)
logging.getLogger().addHandler(fh)

dfknobs, ivals_ref, ivals, gvocs, iknobvals = prepare(MODE)
variables = gvocs.variables

reference_point = {'LT': 3.0, # hours
                   'EFF': 10.0 # % injected
                   }

display(pd.DataFrame(ivals_ref,index=[0]))
display(pd.DataFrame(variables))

,SH1N,SH3N,SH4N,SL1,SL2,SL3,SH1-DW08,SH1-DW18,SH1-DW28,SH3-DW08,SH3-DW18,SH3-DW28,SH4-DW08,SH4-DW18,SH4-DW28
0,3.966582,-1.171022,-3.16418,-2.654321,7.135584,-5.892172,3.966582,3.966582,3.966582,-1.171022,-1.171022,-1.171022,-3.16418,-3.16418,-3.16418


,SH1N,SH3N,SH4N,SL1,SL2,SL3,SH1-DW08,SH1-DW18,SH1-DW28,SH3-DW08,SH3-DW18,SH3-DW28,SH4-DW08,SH4-DW18,SH4-DW28
0,-0.594987,-0.175653,-0.474627,-0.398148,-1.070338,-0.883826,-0.594987,-0.594987,-0.594987,-0.175653,-0.175653,-0.175653,-0.474627,-0.474627,-0.474627
1,0.594987,0.175653,0.474627,0.398148,1.070338,0.883826,0.594987,0.594987,0.594987,0.175653,0.175653,0.175653,0.474627,0.474627,0.474627


In [4]:
TEST_MODE = True
#@TEST_MODE = False

In [5]:
inputs_dict = compute_family_k2l_from_knob_k2l(
    dfknobs, {k:variables[k][0] for k in variables.keys()}, ivals, True)
inputs_dict

SH1N   : +3.967 + -0.595 -> +3.372
SH3N   : -1.171 + -0.176 -> -1.347
SH4N   : -3.164 + -0.475 -> -3.639
SL1    : -2.654 + -0.398 -> -3.052
SL2    : +7.136 + -1.070 -> +6.065
SL3    : -5.892 + -0.884 -> -6.776
SH1-DW08: +3.967 + -0.595 -> +3.372
SH1-DW18: +3.967 + -0.595 -> +3.372
SH1-DW28: +3.967 + -0.595 -> +3.372
SH3-DW08: -1.171 + -0.176 -> -1.347
SH3-DW18: -1.171 + -0.176 -> -1.347
SH3-DW28: -1.171 + -0.176 -> -1.347
SH4-DW08: -3.164 + -0.475 -> -3.639
SH4-DW18: -3.164 + -0.475 -> -3.639
SH4-DW28: -3.164 + -0.475 -> -3.639


{'SH1N': 3.3715950570000004,
 'SH3N': -1.3466749343,
 'SH4N': -3.6388071633,
 'SL1': -3.0524693915000003,
 'SL2': 6.0652466465,
 'SL3': -6.775997940300001,
 'SH1-DW08': 3.3715950570000004,
 'SH1-DW18': 3.3715950570000004,
 'SH1-DW28': 3.3715950570000004,
 'SH3-DW08': -1.3466749343,
 'SH3-DW18': -1.3466749343,
 'SH3-DW28': -1.3466749343,
 'SH4-DW08': -3.6388071633,
 'SH4-DW18': -3.6388071633,
 'SH4-DW28': -3.6388071633}

### Test eval function

In [6]:
# Test
if True and not TEST_MODE:
    raw_eval_f = get_raw_eval_f()
    raw_eval_f(inputs_dict, meas_bxb_tunes=True, meas_lifetime=True, meas_inj_eff=True)

In [7]:
eval_f = get_eval_f(TEST_MODE, gvocs, dfknobs, ivals)
ev = FunctionEvaluator(script=eval_f)

generator_name = 'aps_mobo'
gc = get_generator_class(generator_name)
gen = gc(gvocs=gvocs)
gen.numerical_optimizer.n_raw_samples = 128 #512
gen.numerical_optimizer.n_restarts = 20
gen.gp_constructor.use_low_noise_prior = False
gen.gp_constructor.use_high_noise_prior = False
gen.max_travel_distances = [1.0] * len(gvocs.variables_active)
gen.reference_point = reference_point

[INFO ][18:59:41.516 generator.   16] Initializing generator aps_mobo
[INFO ][18:59:41.517 generator.   64] Initialized generator aps_mobo


In [8]:
options = AlgorithmOptions(store_history=True, dump_every_step=True)
opt = Optimizer(options=options, generator=gen, evaluator=ev, gvocs=gvocs)

[DEBUG][18:59:42.182 optimizer.   83] Initializing optimizer with tag='0de00a4155' store_history=True dump_every_step=True history_directory=None execution_mode='sync'
[DEBUG][18:59:42.183 optimizer.  138] Generator set to: supports_batch_generation=True supports_multi_objective=True vocs=VOCS(variables={'SH1N': [-0.594987363, 0.594987363], 'SH3N': [-0.17565325229999998, 0.17565325229999998], 'SH4N': [-0.4746270213, 0.4746270213], 'SL1': [-0.39814818150000003, 0.39814818150000003], 'SL2': [-1.0703376435, 1.0703376435], 'SL3': [-0.8838258183000001, 0.8838258183000001], 'SH1-DW08': [-0.594987363, 0.594987363], 'SH1-DW18': [-0.594987363, 0.594987363], 'SH1-DW28': [-0.594987363, 0.594987363], 'SH3-DW08': [-0.17565325229999998, 0.17565325229999998], 'SH3-DW18': [-0.17565325229999998, 0.17565325229999998], 'SH3-DW28': [-0.17565325229999998, 0.17565325229999998], 'SH4-DW08': [-0.4746270213, 0.4746270213], 'SH4-DW18': [-0.4746270213, 0.4746270213], 'SH4-DW28': [-0.4746270213, 0.4746270213]}, c

### Sample at initial values

In [9]:
opt.generate_at(iknobvals)
opt.evaluate_data()

[INFO ][18:59:43.671 optimizer.  573] Evaluating 1 inputs
[DEBUG][18:59:43.673 evaluators  193] Data kwargs: row={'SH1N': 0.0, 'SH3N': 0.0, 'SH4N': 0.0, 'SL1': 0.0, 'SL2': 0.0, 'SL3': 0.0, 'SH1-DW08': 0.0, 'SH1-DW18': 0.0, 'SH1-DW28': 0.0, 'SH3-DW08': 0.0, 'SH3-DW18': 0.0, 'SH3-DW28': 0.0, 'SH4-DW08': 0.0, 'SH4-DW18': 0.0, 'SH4-DW28': 0.0}
[DEBUG][18:59:43.674 evaluators  211] Result: output={'result': {'LT': 3.3969010675734883, 'EFF': -0.020352954273539215, '_nux': -1.0382997912282832, '_nuy': 2.037740888877974, '_eps_x_bxbOn': -0.7978345993770948, '_eps_y_bxbOn': -1.2882240120539536}, 'exception': None, 'traceback': '', 'wall_time': 0.0001338999718427658, 'cpu_time': 0.0}
[DEBUG][18:59:43.676 optimizer.  291] Adding (1) new data to internal dataframes
[DEBUG][18:59:43.685 optimizer. 1286] Dumped state to [C:\Users\boss\.apsopt\2023_12_16-18_59_42-aps_mobo-0de00a4155-449c3198\dump_auto_00000.json] in [0.00613]s


In [10]:
assert opt.data.iloc[0].valid, f'ERROR: {opt.data.iloc[0].traceback}'
opt.data

,SH1N,SH3N,SH4N,SL1,SL2,SL3,SH1-DW08,SH1-DW18,SH1-DW28,SH3-DW08,...,traceback,wall_time,cpu_time,valid,LT,EFF,_nux,_nuy,_eps_x_bxbOn,_eps_y_bxbOn
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,,0.000134,0.0,True,3.396901,-0.020353,-1.0383,2.037741,-0.797835,-1.288224


### Sample close to initial point

In [ ]:
mt = {k: 0.05 for k in variables.keys()}
opt.generate_random_around_point(n=7, point=iknobvals, spans=mt, use_normalized_distances=True)
opt.evaluate_data()

In [ ]:
opt.data

### Run optimizer

In [ ]:
opt.step()

In [ ]:
for i in range(20):
    print(f'\n** Step #{i+1} at {datetime.now():%Y-%m-%d %H:%M:%S}**\n')
    opt.step()

In [ ]:
opt.data.tail()

### Plot

In [ ]:
import matplotlib.pyplot as plt

hvs = []
for i in range(1,len(opt.data)):
    hv = opt.generator.calculate_hypervolume_at(i=i)
    hvs.append(hv)

fig,ax = plt.subplots(1,2,squeeze=False,figsize=(12,6))
df = opt.data
ol = ['EFF', 'LT']
l = ax[0,0].scatter(df.loc[:,ol[0]], df.loc[:,ol[1]],10,marker='o',c=range(len(df)),vmax=len(df))
ax[0,0].scatter(df.loc[:1,ol[0]], df.loc[:1,ol[1]], 20,marker='o',c='red',label='initial point')
ax[0,0].scatter(gen.reference_point[ol[0]], gen.reference_point[ol[1]],40,marker='*',c='black',label='reference point')
ax[0,0].set_xlabel(ol[0])
ax[0,0].set_ylabel(ol[1])
fig.colorbar(l)
ax[0,0].legend()

ax[0,1].plot(hvs,marker='o')
ax[0,1].set_xlabel('Step #')
ax[0,1].set_ylabel('Hypervolume')

In [ ]:
fig,ax = plt.subplots(1,2,squeeze=False,figsize=(8,6))
df = opt.data

ax[0,0].plot(df.loc[:,'_nux'], marker='o', label='nux')
ax[0,0].plot(df.loc[:,'_nuy'], marker='o', label='nuy')
ax[0,1].plot(df.loc[:,'_eps_x_bxbOn'], marker='o', label='eps_x_bxbOn', color='red')
ax[0,1].plot(df.loc[:,'_eps_y_bxbOn']*1e2, marker='o', label='eps_y_bxbOn', color='green')
ax[0,0].set_xlabel('Step #')
ax[0,0].set_ylabel('Tune')
ax[0,0].legend()
ax[0,1].legend()

In [ ]:
# This will save optimizer state
if not TEST_MODE:
    opt.state_to_json(f'{datetime.now():%Y%m%dT%H%M%S}_{NAME_ROOT}_dump.json')

In [ ]:
# Nominal (i.e., design) sextupole settings:
opt.generate_at({k:0.0 for k in variables.keys()})
opt.evaluate_data()

In [ ]:
with pd.option_context('display.max_rows', 500):
    display(opt.data)

In [ ]:
opt.generator.model.state_dict()